In [1]:
import psycopg2
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, inspect
import time
from config import host, user, user1, password, password1


In [2]:
start_time = time.time()
postgres_engine = create_engine(f'postgresql://{user1}:{password1}@{host}:5432/staging')
company_name = pd.read_sql('SELECT master_customer_id, company_name, updated_date FROM dw_company_name_dim', con=postgres_engine)

In [ ]:
last_val = company_name['master_customer_id'].sort_values().iloc[-1]
print(last_val)

last_date = company_name['updated_date'].max()
print(last_date)

In [ ]:
stg_bpm = pd.read_sql("SELECT company_name, email FROM stg_bpm_client_info where updated_date >'"+str(last_date)+"'", con=postgres_engine)
stg_blc = pd.read_sql("SELECT publisher_name, email_address FROM stg_blc_customer where role_id = 1 and address_line1 is not null and updated_date >'"+str(last_date)+"'", con=postgres_engine)
master = pd.read_sql("SELECT master_customer_id, email_address FROM dw_master_customer_dim where master_customer_id > '"+str(last_val)+"'", con=postgres_engine)
master

In [ ]:
stg_bpm = stg_bpm.rename(columns={"email": "email_address"})
stg_blc = stg_blc.rename(columns={"publisher_name": "company_name"})
total_data = stg_blc.append(stg_bpm)

In [ ]:
total_data["email_address"] = total_data["email_address"].str.lower()
total_data = total_data[total_data['company_name'] != '']
total_data = total_data.dropna()
print(len(total_data))
len(total_data['email_address'].unique())

In [ ]:
counter = 0

data_df = pd.DataFrame()

for i in total_data['email_address'].unique():

    try:
        if i != None or master['email_address'].str.contains(i).any():
            mc_id = master[master['email_address']==i].to_dict('record')[0]['master_customer_id']
            publisher_name = total_data[total_data['email_address']==i].to_dict('record')[0]['company_name']
            dict_data = {
                    "master_customer_id": mc_id,
                    "company_name": publisher_name
                   }

    #         print(dict_data)


            if dict_data['company_name'] == None or dict_data['company_name'] == '':
                print("Pass")
                pass
            else:
                data_df = data_df.append(dict_data, ignore_index=True)

    except Exception as e:
        print("email:",i)
        counter += 1
        pass
        print("error:",e)
        
    
#     print("----------")

data_df = data_df.drop_duplicates()
print(data_df)
data_df.to_sql(name='dw_company_name_dim', con=postgres_engine, if_exists='append', index=False, method='multi')

display(data_df)

